<a href="https://colab.research.google.com/github/MariaOmaechevarria/OPENCL/blob/main/FILTROS%20IMAGENES/Filtro_sobel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt purge *nvidia* -y
!sudo apt install nvidia-driver-530 -y


!pip install pyopencl
!apt-get install -y pocl-opencl-icd ocl-icd-libopencl1

In [ ]:
import pyopencl as cl
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

from PIL import Image, ImageFilter

In [ ]:
kernel_filter_color_sobel="""
__kernel void kernel_filter_color_sobel(__global uchar* imagen_in,__global uchar* imagen_out,__constant float* filtro_X,__constant float* filtro_Y,int dim,int ancho,int alto){

  int fila = get_global_id(0);
  int columna = get_global_id(1);

  int centro=(dim-1)/2;

  float suma_rojo_X=0.0f;
  float suma_verde_X=0.0f;
  float suma_azul_X=0.0f;
  float suma_rojo_Y=0.0;
  float suma_verde_Y=0.0f;
  float suma_azul_Y=0.0f;

  float T_red=0.0f;
  float T_green=0.0f;
  float T_blue=0.0f;

  int i,j;

  //Asegurarse de que el píxel esté dentro de los límites

   if (centro <= fila && fila < (alto - centro) && centro <= columna && columna < (ancho - centro)) {

       for(i=-centro;i<=centro;i++){

            for(j=-centro;j<=centro;j++){

                // Para acceder al valor del pixel

                int idx = ((fila + i) * ancho + (columna + j)) * 3;

                float pixel_rojo = imagen_in[idx];
                float pixel_verde = imagen_in[idx + 1];
                float pixel_azul = imagen_in[idx + 2];

                float valor_filtro_X = filtro_X[(i + centro) * dim + (j + centro)];
                float valor_filtro_Y = filtro_Y[(i + centro) * dim + (j + centro)];

                suma_rojo_X += pixel_rojo * valor_filtro_X;
                suma_verde_X += pixel_verde * valor_filtro_X;
                suma_azul_X += pixel_azul * valor_filtro_X;

                suma_rojo_Y += pixel_rojo * valor_filtro_Y;
                suma_verde_Y += pixel_verde * valor_filtro_Y;
                suma_azul_Y += pixel_azul * valor_filtro_Y;

                }
        }

        T_red = sqrt(suma_rojo_X * suma_rojo_X + suma_rojo_Y * suma_rojo_Y);
        T_green = sqrt(suma_verde_X * suma_verde_X + suma_verde_Y * suma_verde_Y);
        T_blue = sqrt(suma_azul_X * suma_azul_X + suma_azul_Y * suma_azul_Y);




        int idx_out = (fila * ancho + columna) * 3;
        imagen_out[idx_out] = (uchar)T_red;
        imagen_out[idx_out + 1] = (uchar)T_green;
        imagen_out[idx_out + 2] = (uchar)T_blue;

    }

    else {
        int idx_out = (fila * ancho + columna) * 3;
        imagen_out[idx_out] = imagen_in[idx_out];
        imagen_out[idx_out + 1] = imagen_in[idx_out + 1];
        imagen_out[idx_out + 2] = imagen_in[idx_out + 2];
    }
}
"""

In [ ]:
def importar_imagen(path):
  imagen = Image.open(path)
  return imagen

In [ ]:
def aplicar_filtro_color_sobel(image_path, filtroX,filtroY, kernel_code, kernel_name, device_type, local_size):
    # Importar imagen
    imagen = importar_imagen(image_path)

    # Convertirla a un array de tres canales
    imagen_np = np.array(imagen).astype(np.uint8)

    # Dimensiones de la imagen
    tam_x, tam_y,_= imagen_np.shape

    # Crear array para la imagen final
    imagen_out_np = np.empty_like(imagen_np)

    # Plataforma y dispositivo
    platform = cl.get_platforms()[0]
    device = platform.get_devices(device_type=device_type)[0]

    # Crear contexto y cola de comandos
    context = cl.Context([device])
    command_queue = cl.CommandQueue(context, device=device, properties=cl.command_queue_properties.PROFILING_ENABLE)

    # Crear el programa y compilarlo
    program = cl.Program(context, kernel_code).build()

    # Crear el kernel
    kernel_filter = cl.Kernel(program, kernel_name)

    # Obtener el tamaño máximo del grupo de trabajo
    wg_size = kernel_filter.get_work_group_info(cl.kernel_work_group_info.WORK_GROUP_SIZE, device)

    print(f"Tamaño máximo del grupo de trabajo: {wg_size}")

    # Crear buffers de imagen
    buffer_in = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=imagen_np)
    buffer_out = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, imagen_out_np.nbytes)

    # Crear buffer para el filtro
    filtro_buf_X = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=filtroX)
    filtro_buf_Y = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=filtroY)

    # Argumentos del kernel
    kernel_filter.set_arg(0, buffer_in)
    kernel_filter.set_arg(1, buffer_out)
    kernel_filter.set_arg(2, filtro_buf_X)
    kernel_filter.set_arg(3, filtro_buf_Y)
    kernel_filter.set_arg(4, np.int32(filtroX.shape[0]))
    kernel_filter.set_arg(5, np.int32(imagen_np.shape[1]))
    kernel_filter.set_arg(6, np.int32(imagen_np.shape[0]))

    # Ejecutar el kernel
    global_size = (tam_x, tam_y)
    event = cl.enqueue_nd_range_kernel(command_queue, kernel_filter, global_size, local_size)
    event.wait()

    # Leer el buffer de salida
    cl.enqueue_copy(command_queue, imagen_out_np, buffer_out)

    # Obtener el tiempo de ejecución
    exec_time = 1e-9 * (event.profile.end - event.profile.start)

    # Guardar y mostrar la imagen resultante
    imagen_resultante = Image.fromarray(imagen_out_np)
    imagen_resultante.show()
    #imagen_resultante.save("/content/drive/My Drive/Colab Notebooks/TFG_OPENCL/FILTROS IMAGENES/perrete_sobel.png")

    return imagen_resultante, exec_time


In [ ]:
#FILTRO SOBEL: DETECTA BORDES

filtro_sobel_X=np.array([
    [1, 0, -1],
    [2, 0, -2],
    [1, 0, -1]
], dtype=np.float32)

filtro_sobel_Y=np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype=np.float32)


In [ ]:
def filtro_sobel():
    device_type = cl.device_type.CPU
    image_path = "/content/drive/My Drive/Colab Notebooks/TFG_OPENCL/FILTROS IMAGENES/IMAGENES/pirineos_imagen.png"

    # Definir las combinaciones de local_size, incluyendo (1,1), (2,2), (4,4) y (8,8)
    local_sizes = [(1, 1), (2, 2), (4, 4), (8, 8), (16, 16), (16, 8), (15, 16)]

    # Crear columnas para el DataFrame
    columns = [f"({ls[0]}/{ls[1]})" for ls in local_sizes]
    results_df = pd.DataFrame(index=['Imagen Pirineos'], columns=columns)

    # Probar cada local_size y medir el tiempo de ejecución
    for local_size in local_sizes:
        imagen_resultante, exec_time = aplicar_filtro_color_sobel(image_path, filtro_sobel_X, filtro_sobel_Y, kernel_filter_color_sobel, "kernel_filter_color_sobel", device_type, local_size)

        # Asignar el tiempo de ejecución al DataFrame
        results_df.loc['Imagen Pirineos', f"({local_size[0]}/{local_size[1]})"] = exec_time if exec_time is not None else "NP"
        print(f"local_size: {local_size}, tiempo de ejecución: {exec_time}")

    # Guardar los resultados en un archivo CSV
    results_df.to_csv('/content/drive/My Drive/Colab Notebooks/TFG_OPENCL/FILTROS IMAGENES/Filtros_sobel_CPU.csv', index=True)

filtro_sobel()
